# # 지도를 통한 제주도 금융 스타일
지도를 활용하여 제주도민의 금융 스타일을 시각화하는 것을 시도하였습니다.

### 패키지 로드

In [1]:
# 기본
import json
import pandas as pd
import os

In [2]:
# interactive widge을 위한 패키지
from IPython.display import display
from ipywidgets import interactive
from ipywidgets import widgets

In [3]:
# folium: 지도
import folium

In [4]:
# plotly: 그래프 패키지
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [5]:
from IPython.display import clear_output
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from folium.plugins import HeatMap , HeatMapWithTime
import pickle

### 데이터 로드

In [6]:
jeju_data =  pd.read_csv("../../data/jeju_financial_life_data.csv")

### 데이터 생성 및 처리

In [10]:
# 제주도 우편 번호를 동으로 변환해 놓은 zip_dict 파일 로드
with open('../../data/zip_dict.pkl', 'rb') as f:
    zip_dict = pickle.load(f)

In [11]:
# 동이름을 'town_name'에 저장
jeju_data['town_name'] = jeju_data['zip_cd'].replace(zip_dict)

In [12]:
# -999,999가 수치 계산에 영향을 미치지 못 하도록 nan으로 처리
jeju_data.loc[jeju_data['medium_resid_rat']<0, 'medium_resid_rat'] = np.nan
jeju_data.loc[jeju_data['large_resid_rat']<0, 'large_resid_rat'] = np.nan

### 데이터 분석

In [13]:
val_cols = ['avg_income', 'med_income', 'avg_spend',
       'avg_foreign_spend', 'avg_debt', 'avg_debt_credit', 'avg_debt_noneb',
       'avg_debt_mortgage', 'avg_debt_deposit', 'avg_debt_collateral',
       'avg_credit_rat', 'medium_resid_rat', 'large_resid_rat',
       'vehicle_own_rat',  'job_majorc',
       'job_smallc', 'job_public', 'job_profession', 'job_self', 'job_none',
       'job_other']
job_cols = ['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self', 'job_none']

### 시각화

In [14]:
# 지도 타입 위젯
map_buttons = widgets.ToggleButtons(
    options=[('기본', None), ('흑', 'cartodbdark_matter'), ('흑백', 'stamentoner')],
    description='지도 배경:',
)
# 수치 계산 위젯
metric_buttons = widgets.ToggleButtons(
    options=[('합계', 'sum'), ('평균', 1)],
    description='계산 방법:',
)
# 수치 선택 위젯
val_buttons = widgets.Dropdown(
    options=val_cols,
    description='수치:',
)
# 성별 선택 위젯
sex_buttons = widgets.ToggleButtons(
    options=[('전체', 0), ('남', 1), ('여', 2)],
    description='성별:',
)
# 직업 선택 위젯
job_buttons = widgets.ToggleButtons(
    options=[('전체', 0), ('대기업', 'job_majorc'), ('중소기업', 'job_smallc'), ('공무원', 'job_public'), ('자영업', 'job_profession'), ('무직', 'job_self'), ('기타', 'job_none')],
    description='직업:',
)
# 신용등급 선택 위젯
options = range(2,7+1)
credit_sliders  = widgets.SelectionRangeSlider(
    options=options,
    index=(0,5),
    description='신용등급',
    disabled=False
)
# 연령대 선택 위젯
options = range(24, 99+1, 5)
age_sliders = widgets.SelectionRangeSlider(
    options=options,
    index=(0,len(options)-1),
    description='연령대',
    disabled=False,
)

# 위젯을 container로 합침.
container = widgets.VBox([
                widgets.HBox([map_buttons]),
                widgets.HBox([metric_buttons]),
                widgets.HBox([val_buttons]),
                widgets.HBox([sex_buttons]),
                widgets.HBox([job_buttons]),
                widgets.HBox([credit_sliders]),
                widgets.HBox([age_sliders]),
                ])

In [15]:
def draw_jeju_map_val(df, col, metric, map_type):
    # 제주도 지도
    rome_lat, rome_lng = 33.379894, 126.545211
    m = folium.Map(location=[rome_lat, rome_lng], zoom_start=10) #, height='50%')

    # 수치 계산 방법
    if metric=='sum':
        tmp_df = df.groupby(['town_name'], as_index=False).mean()
        df = df.groupby(['town_name'], as_index=False).sum()
        df.loc[:, ['x_axis', 'y_axis', 'sex', 'age', 'town_name']] = tmp_df.loc[df.index, ['x_axis', 'y_axis', 'sex', 'age', 'town_name']]
        del tmp_df
    else: # metric=='mean':
        df = df.groupby(['town_name'], as_index=False).mean()
    
    # 표시 단위를 적절히 조정하기 위해 MinMaxScaler 적용하여 컬럼 col 값 업데이트
    tmp = df[col].values.reshape(-1,1)
    scaler = MinMaxScaler(feature_range=(4, 7))
    scaler.fit(tmp)
    df['scaled_'+col] = scaler.transform(tmp).reshape(-1)
    
    # 지도에 각 데이터 당 circle 생성
    for i in df.index.tolist():
        row = df.loc[i]
        radius = np.exp(row['scaled_'+col])
        #print ('radius : {0}'.format(radius))
        folium.Circle(
            location=[row['y_axis'], row['x_axis']],
            tooltip='{0}: {1:,.4f}'.format(row['town_name'], row[col]),
            radius=radius,
            #color='crimson',
            fill=True,
            #fill_color='crimson',
            line_weight=0.01,
        ).add_to(m)
    
    # 지도 종류 선택
    if map_type is None:
        # Nothing
        None
    elif map_type=='cartodbdark_matter':
        folium.TileLayer('cartodbdark_matter').add_to(m)
    elif map_type=='stamentoner':
        folium.TileLayer('stamentoner').add_to(m)
    return m #display(m)

In [16]:
def response(change):
    # 위젯에 변화가 감지되면 실행되는 코드

    # 이전 셀 아웃풋은 지움.
    clear_output()  
    
    temp_df = jeju_data.copy(deep=True) 
    
    # 성별
    if sex_buttons.value != 0:
        temp_df = temp_df[temp_df.sex==sex_buttons.value]
        
    # 신용 등급
    min_rat, max_rat = credit_sliders.value
    if min_rat==2 and max_rat==7:
        # not select
        None
    elif min_rat==max_rat:
        temp_df = temp_df[temp_df.avg_credit_rat==min_rate]
    else:
        temp_df = temp_df[(temp_df.avg_credit_rat>min_rat) & (temp_df.avg_credit_rat<=max_rat)]

    
    # 연령
    min_age, max_age = age_sliders.value
    if min_age==24 and max_age==99:
        # not select
        None
    elif min_age==max_age:
        temp_df = temp_df[temp_df.age==min_age]
    else:
        temp_df = temp_df[(temp_df.age>min_age) & (temp_df.age<=max_age)]
    
        
    # 수치, 수치가 0인 row는 제외
    col = val_buttons.value
    temp_df = temp_df[temp_df[col]!=0]
    if col=='medium_resid_rat' or col=='large_resid_rat':
        temp_df.dropna(inplace=True)
    
    # 직업별 수치, 직업이 0인 row는 제외
    if job_buttons.value != 0:
        temp_df = temp_df[temp_df[job_buttons.value] != 0]
        temp_df[col] *= temp_df[job_buttons.value]
    
    my_map = draw_jeju_map_val(temp_df, col, metric_buttons.value, map_buttons.value)
    
    # 위젯 출력
    display(container)
    print ('선택된 그룹은 전체 그룹 중 {0:.1f}% ({1:d}명)를 차지합니다.'.format(len(temp_df)/len(jeju_data)*100, len(temp_df)))
    # 지도 출력
    display(my_map)

In [17]:
# 위젯 변화를 observe하여 response 함수를 실행
sex_buttons.observe(response, names="value")
credit_sliders.observe(response, names="value")
val_buttons.observe(response, names="value")
job_buttons.observe(response, names="value")
age_sliders.observe(response, names="value")
metric_buttons.observe(response, names="value")
map_buttons.observe(response, names="value")

## 인터렉티브 데모
버튼 혹은 드랍다운 메뉴에서 항목을 선택할 경우, 선택한 집단에 따른 수치가 그래프로 표시됩니다. <br />
**수치**에 있는 'job_' 항목을 선택하면 해당 직업을 가진 주민들이 어디에 많이 거주하고 있는 지를 표시합니다. <br />
**직업**에 있는 항목을 선택하면 해당 직업을 가진 데이터를 선택합니다.

In [18]:
display(container)
my_map = draw_jeju_map_val(jeju_data, val_buttons.value, metric_buttons.value, map_buttons.value)
display(my_map)

선택된 그룹은 전체 그룹 중 99.4% (10358명)를 차지합니다.
